## Data

In [110]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from raise_utils.data import Data
import gc

In [3]:
df = pd.read_csv('./Bug-Related-Activity-Logs/firefox.csv')
df.drop(['Unnamed: 0', 'bugID'], axis=1, inplace=True)

_df = df[['s1', 's2', 's3', 's4', 's5', 's6', 's8', 'y']]
_df['s70'] = df['s7'].apply(lambda x: eval(x)[0])
_df['s71'] = df['s7'].apply(lambda x: eval(x)[1])
_df['s72'] = df['s7'].apply(lambda x: eval(x)[2])

_df['s90'] = df['s9'].apply(lambda x: eval(x)[0])
_df['s91'] = df['s9'].apply(lambda x: eval(x)[1])
_df['s92'] = df['s9'].apply(lambda x: eval(x)[2])

x = _df.drop('y', axis=1)
y = _df['y']

data = Data(*train_test_split(x, y))
data.y_train = data.y_train < 4
data.y_test = data.y_test < 4

<ipython-input-3-c86471ff691a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['s70'] = df['s7'].apply(lambda x: eval(x)[0])
<ipython-input-3-c86471ff691a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df['s71'] = df['s7'].apply(lambda x: eval(x)[1])
<ipython-input-3-c86471ff691a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [7]:
df.head()

,user_comments,system_records,s1,s2,s3,s4,s5,s6,s7,s8,s9,y
0,"['eyedropper', 'style', 'editor', 'use', 'shar...",[],1,1,0,32,1,0,"[0, 0, 0]",1,"[0, 1, 0]",3
1,"['258347', 'assignee', 'comment1', 'createdatt...","['assignee', 'nobody', 'archaeopteryx', 'statu...",13,6,7,260,2,2,"[0, 1, 0]",3,"[0, 1, 1]",1
2,"['75935', 'comment7']","['keywords', 'checkin-needed', 'whiteboard', '...",2,1,1,10,3,3,"[1, 0, 1]",1,"[0, 0, 1]",0
3,"['keyboard', 'long', 'word', 'suggestions', 'c...","['see', 'also', 'bug', 'blocking-b2g', '2.0', ...",5,2,3,81,1,0,"[0, 0, 0]",2,"[1, 0, 1]",9
4,"['465859', 'comment2', 'ni', 'william', 'regre...","['cc', 'bhuang', 'whsu', 'flags', 'needinfo', ...",2,1,1,27,2,3,"[1, 0, 0]",1,"[0, 0, 1]",6


## Word embeddings

In [82]:
from gensim.models.fasttext import FastText

In [76]:
x_user_raw = [eval(x) for x in df['user_comments']]
x_user_raw_train, x_user_raw_test = train_test_split(x_user_raw)

In [83]:
user_model = FastText(sentences=x_user_raw_train, sg=1, size=200)

In [79]:
x_sys_raw = [eval(x) for x in df['system_records']]
x_sys_raw_train, x_sys_raw_test = train_test_split(x_sys_raw)

In [84]:
system_model = FastText(sentences=x_sys_raw_train, sg=1, size=200)

In [86]:
user_model['boooooo'].shape

<ipython-input-86-77f8e53d42cb>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  user_model['boooooo'].shape


(200,)

## Activity log stream encoding

In [149]:
from keras.layers import Bidirectional, LSTM, Dense, BatchNormalization, merge, Input, LeakyReLU, Flatten, Reshape
from keras.preprocessing.sequence import pad_sequences

In [99]:
maxlen = 50
x_user_train = [t[:50] if len(t) >= 50 else t.extend(['end'] * (50 - len(t))) for t in x_user_raw_train]

In [101]:
x_user_train = [t for t in x_user_train if t is not None]

In [102]:
np.array(x_user_train).shape

(20193, 50)

In [103]:
x_sys_train = [t[:50] if len(t) >= 50 else t.extend(['end'] * (50 - len(t))) for t in x_sys_raw_train]
x_sys_train = [t for t in x_sys_train if t is not None]

In [105]:
x_user_train = [[user_model[word] for word in arr] for arr in x_user_train]

<ipython-input-105-ad4210106d98>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  x_user_train = [[user_model[word] for word in arr] for arr in x_user_train]


In [107]:
x_user_train = np.array(x_user_train)

In [108]:
x_sys_train = np.array([[system_model[word] for word in arr] for arr in x_sys_train])

<ipython-input-108-09af2f8763fd>:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  x_sys_train = np.array([[system_model[word] for word in arr] for arr in x_sys_train])


In [109]:
x_user_train.shape, x_sys_train.shape

((20193, 50, 200), (769, 50, 200))

In [111]:
x_user_test = [t[:50] if len(t) >= 50 else t.extend(['end'] * (50 - len(t))) for t in x_user_raw_test]
x_user_test = [t for t in x_user_test if t is not None]

In [112]:
x_sys_test = [t[:50] if len(t) >= 50 else t.extend(['end'] * (50 - len(t))) for t in x_sys_raw_test]
x_sys_test = [t for t in x_sys_test if t is not None]

In [113]:
del x_sys_raw_train, x_sys_raw_test, x_user_raw_train, x_user_raw_test
gc.collect()

10620

In [121]:
user_activity_input = Input(shape=x_user_train.shape[1:])

user_activity_stream_hidden_layer = Bidirectional(
    LSTM(128, return_sequences=True, input_shape=(x_user_train.shape[1], x_user_train.shape[2]))
)(user_activity_input)

user_activity_stream = Bidirectional(
    LSTM(256, return_sequences=True, input_shape=(x_user_train.shape[1], 256))
)(user_activity_stream_hidden_layer)

In [122]:
sys_activity_input = Input(shape=x_sys_train.shape[1:])

sys_activity_stream_hidden_layer = Bidirectional(
    LSTM(128, return_sequences=True, input_shape=x_sys_train.shape[1:])
)(sys_activity_input)

sys_activity_stream = Bidirectional(
    LSTM(32, return_sequences=True, input_shape=(x_sys_train.shape[1], 256))
)(sys_activity_stream_hidden_layer)

In [126]:
meta_input = Input(shape=data.x_train.shape[1:])

meta_hidden_layer = Dense(50)(meta_input)
meta_hidden_activation = LeakyReLU()(meta_hidden_layer)

meta = Dense(30)(meta_hidden_activation)
meta = LeakyReLU()(meta)

## Concatenate

In [138]:
flatten1 = Flatten()(user_activity_stream)
flatten2 = Flatten()(sys_activity_stream)

concat_layer = merge.Concatenate(axis=-1)([flatten1, flatten2, meta])

## Merging MLP layer

In [139]:
mlp_layer1 = Dense(380, input_shape=())(concat_layer)
mlp_activ1 = LeakyReLU()(mlp_layer1)

mlp_layer2 = Dense(450)(mlp_activ1)
mlp_activ2 = LeakyReLU()(mlp_layer2)

mlp_layer3 = Dense(260)(mlp_activ2)
mlp_activ3 = LeakyReLU()(mlp_layer3)

mlp_layer4 = Dense(200)(mlp_activ3)
mlp_activ4 = LeakyReLU()(mlp_layer4)

mlp_out = Dense(200)(mlp_activ4)
mlp_out = LeakyReLU()(mlp_out)

## Bin-sequence embedding

In [157]:
# from https://gist.github.com/skeeet/b639eea7e3fc51dd03e9b69c06b2fdf1
def make_residual_lstm_layers(input, rnn_width, rnn_depth):
    """
    The intermediate LSTM layers return sequences, while the last returns a single element.
    The input is also a sequence. In order to match the shape of input and output of the LSTM
    to sum them we can do it only for all layers but the last.
    """
    x = input
    for i in range(rnn_depth):
        return_sequences = i < rnn_depth - 1
        x_rnn = LSTM(rnn_width, return_sequences=return_sequences)(input)
        x_rnn = BatchNormalization()(x_rnn)
        if return_sequences:
            # residual block
            x = merge.Add()([x, x_rnn])
        else:
            # last layer does not return sequences and cannot be residual
            x = x_rnn
    return x

In [158]:
reshape = Reshape((1, 200))(mlp_out)

In [159]:
dasenet = make_residual_lstm_layers(reshape, 200, rnn_depth=8)

In [161]:
dasenet.shape

TensorShape([None, 200])

In [166]:
n_classes = 2
final_layer = Dense(n_classes, activation='softmax')(dasenet)

In [163]:
from keras.models import Model

In [167]:
model = Model(inputs=[user_activity_input, sys_activity_input, meta_input], outputs=[final_layer])

In [168]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 50, 200)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 50, 200)]    0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 50)           700         input_3[0][0]                    
______________________________________________________________________________________________